# 合并表格

In [1]:
import numpy as np
import pandas as pd

# ### 读取
# trd_dalyr0=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr.xlsx")
# trd_dalyr0=trd_dalyr0.drop([0,1],axis=0)
# trd_dalyr1=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr1.xlsx")
# trd_dalyr1=trd_dalyr1.drop([0,1],axis=0)
# trd_dalyr2=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr2.xlsx")
# trd_dalyr2=trd_dalyr2.drop([0,1],axis=0)
# trd_dalyr3=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr3.xlsx")
# trd_dalyr3=trd_dalyr3.drop([0,1],axis=0)
# trd_dalyr4=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr4.xlsx")
# trd_dalyr4=trd_dalyr4.drop([0,1],axis=0)
# diaryData=[trd_dalyr0,trd_dalyr1,trd_dalyr2,trd_dalyr3,trd_dalyr4]
# finalDF=pd.concat(diaryData)
# finalDF

In [2]:
# companyData=pd.read_excel("stockData/furtherInformation/公司文件184301535(仅供中央财经大学使用)/TRD_Co.xlsx")
# companyData=companyData.drop([0,1],axis=0)
# companyData

In [3]:
# companyAssetData=pd.read_excel("stockData/furtherInformation/资产负债表(联表查询)181028530/FS_Combas(Merge Query).xlsx")
# companyAssetData=companyAssetData.drop([0,1],axis=0)
# companyAssetData

In [4]:
# ### 去除b部分
# companyAssetData=companyAssetData[companyAssetData["FS_Combas.Typrep"]!="B"]
# companyAssetData=companyAssetData.reindex()
# ### 去除12-31，因为与1-1有重复
# companyAssetData=companyAssetData[companyAssetData['FS_Combas.Accper'].str.contains("01-01|03-31|06-30|09-30")]
# companyAssetData['FS_Combas.Accper']=pd.to_datetime(companyAssetData['FS_Combas.Accper'], format="%Y-%m-%d")
# 
# companyAssetData=companyAssetData.rename(columns={"FS_Combas.Stkcd":"Stkcd","FS_Combas.Accper":"Trddt_quarter"})
# companyAssetData

In [5]:
# ### todo：开始合并 日个股回报与公司信息
# df=pd.merge(finalDF,companyData,on='Stkcd')
# df['Trddt']=pd.to_datetime(df["Trddt"], format="%Y-%m-%d")
# df

In [6]:
###转换时间为季度
def dateToQuarter(date):
        # print("time: ",date)
        quarter_date={1:"01-01",2:"03-31",3:"06-30",4:"09-30"}
        year=date.astype('datetime64[Y]').astype(int) + 1970
        month = date.astype('datetime64[M]').astype(int) % 12 + 1
        the_quarter = (month - 1) // 3 + 1                  #计算季度
        quarter_name = str(year) + '年' + str(the_quarter) + '季度'
        quarter_name_short = str(year) + 'Q' + str(the_quarter)
        # print("quarter time: ",quarter_name)
        return [str(year)+"-"+quarter_date[the_quarter],the_quarter]

# dateToQuarter(numpy.datetime64("2020-04-30"))

In [7]:
# df["Trddt_quarter"]=[dateToQuarter(i)[0] for i in df['Trddt'].values]
# df["Trddt_quarter"]=pd.to_datetime(df["Trddt_quarter"], format="%Y-%m-%d")
# df

In [8]:
# ###todo：根据季度与股票代码合并之前的总表与资产负债表
# allData=pd.merge(df,companyAssetData,on=["Stkcd",'Trddt_quarter'],how='outer')
# # allData['Stkcd']=allData['Stkcd'].astype(str)
# allData

In [9]:
# allData.to_csv("stockData/furtherInformation/AllData.csv")

# 数据去重，并且删除部分缺失值

In [10]:
# allData=pd.read_csv("stockData/furtherInformation/AllData.csv",index_col=0,dtype=object)
# allData['Trddt']=pd.to_datetime(allData["Trddt"], format="%Y-%m-%d")
# allData

In [11]:
# ###todo：要删除的重复列名
# deleteColumns=["Trddt_quarter","FS_Combas.Typrep","FS_Combas.ShortName","csmar_listedcoinfo.Stknme","csmar_listedcoinfo.Conme","FS_Comscfd.ShortName","FS_Comins.ShortName","csmar_listedcoinfo.Conme_en","csmar_listedcoinfo.Nnindcd","csmar_listedcoinfo.PROVINCECD","csmar_listedcoinfo.CITY","csmar_listedcoinfo.Nnindnme"]
# afterDeleteData=allData[[i for i in allData.columns if i not in deleteColumns]]
# afterDeleteData

In [12]:
# afterDeleteData.to_csv("stockData/furtherInformation/afterDeleteData.csv")

In [13]:
# ### 检查是否有缺失值达到50%的，有的话就删除
# totalNumRows=5000412
# num = afterDeleteData.isna().sum()
# indexs=[]
# nullNums=[]
# nullPerc=[]
# for index in num.index:
#     indexs.append(index)
#     nullNums.append(num[index])
#     nullPerc.append(str(round(float(num[index]/totalNumRows),3)*100)+" %")
# 
# num=pd.DataFrame.from_dict({"Variance":indexs,"Number Of Null":nullNums,"Percentage Of Null":nullPerc})
# num

In [14]:
# ###todo：删除和处理回归不能输入的数据,删除回报率为null的行,同时删除缺失值过多的 [FS_Combas.A001127000,FS_Combas.A001220000,FS_Combas.A001219000,FS_Combas.A001109000] 列
# uselessColumns=["FS_Combas.A001127000","FS_Combas.A001220000","FS_Combas.A001219000","FS_Combas.A001109000","Stknme","Listdt","csmar_listedcoinfo.BUSSINESSRANGE","Conme_en","PROVINCECODE","Parvcur","csmar_listedcoinfo.MAINBUSSINESS","CITYCODE","PROVINCE","Cuntrycd","Conme","Nindcd","Nindcd","Nnindcd","OWNERSHIPTYPECODE","Opnprc","Hiprc","Loprc","Clsprc","Indcd","csmar_listedcoinfo.OWNERSHIPTYPE","Markettype","Sctcd"]
# 
# uselessDeletedDf=allData[[i for i in allData.columns if i not in deleteColumns and i not in uselessColumns]]
# uselessDeletedDf=uselessDeletedDf.dropna()
# uselessDeletedDf

In [15]:
###查看并确定目前的数据类型
# print("占有类型： ",uselessDeletedDf["OWNERSHIPTYPE"].unique())
# print("财产类型：",uselessDeletedDf["csmar_listedcoinfo.EquityNature"].unique())
# print("产业类型：",uselessDeletedDf["Indnme"].unique())
# print("产业细分1：",uselessDeletedDf["Nindnme"].unique())
# print("产业细分2：",uselessDeletedDf["Nnindnme"].unique())
# print("公司所在地区：",uselessDeletedDf["CITY"].unique())

In [16]:
# uselessDeletedDf.to_csv("stockData/furtherInformation/Input.csv",index=False)

# 数据转换 与 回归

In [17]:
uselessDeletedDf=pd.read_csv("stockData/furtherInformation/Input.csv")

In [18]:
### 使用onehot 与 lable转换一些数据
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
uselessDeletedDf['Trddt']=pd.to_datetime(uselessDeletedDf["Trddt"], format="%Y-%m-%d")
start = datetime(2019, 1, 1)
end = datetime(2020, 1, 1)
uselessDeletedDf=uselessDeletedDf[(uselessDeletedDf['Trddt'] < end) | (uselessDeletedDf['Trddt'] > start)]
###增加一个季度变量
lableEncoder=LabelEncoder()
uselessDeletedDf['Quarter']=[dateToQuarter(i)[1] for i in uselessDeletedDf['Trddt'].values]
Trddt_lable=lableEncoder.fit_transform([[x] for x in uselessDeletedDf['Trddt'].values])
uselessDeletedDf['Trddt']=Trddt_lable
# Stkcd_lable=lableEncoder.fit_transform([[x] for x in uselessDeletedDf['Stkcd'].values])
# uselessDeletedDf['Stkcd']=Trddt_lable

encoder = OneHotEncoder(sparse=False)
OWNERSHIPTYPE_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['OWNERSHIPTYPE'].values])
EquityNature_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['csmar_listedcoinfo.EquityNature'].values])
Indnme_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['Indnme'].values])
Nindnme_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['Nindnme'].values])
Nnindnme_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['Nnindnme'].values])
CITY_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['CITY'].values])
# uselessDeletedDf['Stkcd']=uselessDeletedDf['Stkcd'].astype(int)
# Stkcd_onehot = encoder.fit_transform([[x] for x in uselessDeletedDf['Stkcd'].values])
# Stkcd_onehot

C:\Users\87066\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\87066\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\87066\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\87066\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was re

In [19]:
def addColumnToDataFrame(name:str, theArray, theDataFrame=uselessDeletedDf):
    theDict={}
    thelist = [[r[col] for r in theArray] for col in range(len(theArray[0]))]
    # print(f"{name} column: {len(thelist)}")
    # print(f"{name} row: {[len(a) for a in Color_onehot][0]}")
    numOfColumn = 0
    for column in thelist:
        theDict[name + str(numOfColumn)] = column
        numOfColumn += 1
    newDF=pd.DataFrame.from_dict(theDict)
    theDataFrame=pd.concat([theDataFrame,newDF])
    # return theDataFrame

In [20]:
addColumnToDataFrame('OWNERSHIPTYPE',OWNERSHIPTYPE_onehot)
addColumnToDataFrame('csmar_listedcoinfo.EquityNature',EquityNature_onehot)
addColumnToDataFrame('Indnme',Indnme_onehot)
addColumnToDataFrame('Nindnme',Nindnme_onehot)
addColumnToDataFrame('Nnindnme',Nnindnme_onehot)
addColumnToDataFrame('CITY',CITY_onehot)
# addColumnToDataFrame('Stkcd',Stkcd_onehot)
uselessDeletedDf=uselessDeletedDf.drop(columns=['OWNERSHIPTYPE','csmar_listedcoinfo.EquityNature','Indnme','Nindnme','Nnindnme','CITY','Stkcd'])
uselessDeletedDf

MemoryError: Unable to allocate 3.84 GiB for an array with shape (69, 7476204) and data type float64

In [ ]:
from sklearn.model_selection import train_test_split

### train & test data
Y=uselessDeletedDf['ChangeRatio']
X=uselessDeletedDf.drop(columns='ChangeRatio')
x_train , x_test , y_train , y_test = train_test_split(X,Y , test_size= 0.3 ,random_state=42)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train)

In [ ]:
y_pred = linear_regressor.predict(x_test)
y_trainPredicted=linear_regressor.predict(x_train[-3000:])
LR = pd.DataFrame({'y_test':y_test,'y_pred':y_pred})
LR

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12,8))
plt.plot(LR[:50])
plt.legend(['Actual' , 'Predicted'])